# Dashboards Deep Dive: Parameters and Filters

Build interactive dashboards with dynamic parameters and filters using your semantic model.

## What You'll Learn

✅ Create static parameters for quick dropdown filters  
✅ Use query-based parameters for dynamic values  
✅ Add interactive filters to dashboards  
✅ Build visualizations from semantic models  

---

## Why Interactive Dashboards?

Your leadership needs dashboards ready by end of week. Basic static charts won't cut it - they need:
- **Filters by factory and region** - Different stakeholders need different views
- **Date range selection** - View last 7 days, 30 days, or custom ranges
- **Dynamic updates** - As new data arrives, dashboards auto-refresh

**Parameters and filters** make this possible without creating dozens of separate dashboards.

---

## Setup: Use Your Semantic Model

In the previous notebook, you created the `iot_sensor_metrics` semantic model. This dashboard will query that model, ensuring consistent metrics across all visualizations.

**Reference:**
- Table: `default.db_crash_course.iot_sensor_metrics`
- Metrics: `average_temperature`, `active_devices`, `total_sensor_readings`
- Dimensions: `factory_name`, `region`, `model_category`, `reading_date`

---

**References:**
- [Dashboard Parameters](https://docs.databricks.com/aws/en/dashboards/tutorials/query-based-params)
- [Dashboard Filters](https://docs.databricks.com/aws/en/dashboards/filters)
- [Creating Dashboards](https://docs.databricks.com/aws/en/dashboards/)


## 1. Creating a Dashboard with Parameters

### Step 1: Create a New Dashboard

1. Click **Dashboards** in the sidebar
2. Click **Create Dashboard**
3. Name it: `IoT Factory Performance`
4. Click **Create**

### Step 2: Add Your First Query

Click **Add** → **Visualization**

---

## 2. Static Parameters - Quick Setup

**Static parameters** are hardcoded lists - fastest to set up for fixed options.

### Example: Region Filter

**Query with Static Parameter:**


In [0]:
# Query: factory_temperature_by_region
# This query will be used as a dataset in your Databricks Dashboard

query_with_static_param = """
SELECT 
  factory_name,
  region,
  average_temperature,
  active_devices,
  total_sensor_readings
FROM default.db_crash_course.iot_sensor_metrics
WHERE region = '{{region_filter}}'
ORDER BY average_temperature DESC
"""

print("Query with static parameter:")
print(query_with_static_param)
print("\n📋 In your dashboard: Click 'Add' → 'Visualization' → paste this query")


In [0]:
-- Factory Performance Summary using metric view
-- 
-- IMPORTANT: Replace YOUR_SCHEMA with your personal schema name
-- (extract username before @ symbol)
-- Example: if your email is jane.smith@company.com, use: jane_smith

SELECT 
  `Factory Name`,
  `Region`,
  `Average Temperature`,
  `Active Devices`,
  `Total Sensor Readings`
FROM dwx_airops_insights_platform_dev_workspace.YOUR_SCHEMA.iot_sensor_metrics
WHERE `Reading Date` >= CURRENT_DATE - 30
ORDER BY `Average Temperature` DESC

### Query 2: Temperature Trend by Region


In [0]:
-- Temperature trends over time grouped by region
-- IMPORTANT: Replace YOUR_SCHEMA with your personal schema name

SELECT 
  `Reading Date`,
  `Region`,
  AVG(`Average Temperature`) as avg_temp,
  SUM(`Total Sensor Readings`) as total_readings
FROM dwx_airops_insights_platform_dev_workspace.YOUR_SCHEMA.iot_sensor_metrics
WHERE `Reading Date` >= CURRENT_DATE - 30
GROUP BY `Reading Date`, `Region`
ORDER BY `Reading Date`, `Region`

### Query 3: Critical Temperature Count by Model Category


In [0]:
-- Count of critical readings by model category
-- IMPORTANT: Replace YOUR_SCHEMA with your personal schema name

SELECT 
  `Model Category`,
  SUM(`Critical Reading Count`) as critical_readings,
  SUM(`Total Sensor Readings`) as total_readings,
  ROUND(100.0 * SUM(`Critical Reading Count`) / SUM(`Total Sensor Readings`), 2) as critical_pct
FROM dwx_airops_insights_platform_dev_workspace.YOUR_SCHEMA.iot_sensor_metrics
WHERE `Reading Date` >= CURRENT_DATE - 7
GROUP BY `Model Category`
ORDER BY critical_pct DESC

**💡 Key Observations:**
- Metric views use business-friendly names (backticks for names with spaces)
- Joins are automatic (no need to manually join `dim_factories` or `dim_models`)
- Metrics like `Average Temperature` and `Critical Reading Count` are pre-calculated
- These queries can be used directly in dashboards or Genie spaces

---

## 3. Query-Based Parameters - Dynamic Lists

Now let's create dynamic parameters that populate from the metric view.

### Step 1: Create Parameter Query

Create a new query to fetch factory names from the metric view:


### Creating the Static Parameter

**In the SQL Editor:**

1. Paste the query above
2. Notice `{{region_filter}}` is highlighted
3. Click on `{{region_filter}}`
4. Select **Create Parameter**
5. Configure:
   - **Type**: Dropdown list
   - **Values**: 
     ```
     West
     East
     Central
     ```
   - **Default**: West
6. Click **Save**

**Result:** A dropdown appears above your query. Change the region to see different results!

### Add Visualization

**Bar Chart:**
- **X-axis**: `factory_name`
- **Y-axis**: `average_temperature`
- **Chart type**: Bar chart
- **Title**: Factory Temperature by Region

---

## 3. Query-Based Parameters - Dynamic Lists

**Query-based parameters** populate from live data - always up to date as new factories or models are added.

### Step 1: Create Parameter Query

Create a new query to fetch factory names:


In [0]:
# Query: get_factory_list
# This populates the dropdown dynamically

parameter_query = """
SELECT DISTINCT 
  factory_name as value,
  factory_name as label
FROM default.db_crash_course.iot_sensor_metrics
ORDER BY factory_name
"""

print("Parameter query for factory dropdown:")
print(parameter_query)
print("\n📋 Copy this query into a new SQL query named 'get_factory_list'")


### Step 2: Create Main Query Using Parameter

Now create a query that uses the factory parameter:


In [0]:
# Query: factory_performance_trend
# Uses the query-based parameter

query_with_dynamic_param = """
SELECT 
  reading_date,
  factory_name,
  average_temperature,
  active_devices
FROM default.db_crash_course.iot_sensor_metrics
WHERE factory_name = '{{factory_param}}'
  AND reading_date >= CURRENT_DATE - INTERVAL 30 DAYS
ORDER BY reading_date
"""

print("Query with query-based parameter:")
print(query_with_dynamic_param)
print("\n📋 Copy this query into a new SQL query in Databricks SQL")


### Step 3: Link Parameter to Query

**In the SQL Editor:**

1. Click on `{{factory_param}}`
2. Select **Create Parameter**
3. Configure:
   - **Type**: Query Based Dropdown List
   - **Query**: Select `get_factory_list` (the query you just created)
   - **Title**: Factory Name
4. Click **Save**

**Result:** Dropdown automatically populated with all factory names from your data!

### Add Visualization

**Line Chart:**
- **X-axis**: `reading_date`
- **Y-axis**: `average_temperature`
- **Chart type**: Line chart
- **Title**: Temperature Trend by Factory (30 Days)

---

## 4. Dashboard Filters - Quick Examples

Filters apply across all queries in the dashboard.

### Filter 1: Date Range

Create a query that uses a date range filter:


In [0]:
# Query: readings_summary
# Uses date range filter

query_with_date_filter = """
SELECT 
  factory_name,
  region,
  SUM(total_sensor_readings) as total_readings,
  AVG(average_temperature) as avg_temp,
  MAX(active_devices) as max_devices
FROM default.db_crash_course.iot_sensor_metrics
WHERE reading_date BETWEEN '{{start_date}}' AND '{{end_date}}'
GROUP BY factory_name, region
ORDER BY total_readings DESC
"""

print("Query with date range filter:")
print(query_with_date_filter)
print("\n📋 Copy this query into a new SQL query")


**Setup Date Parameters:**

1. Click on `{{start_date}}`
2. **Type**: Date
3. **Default**: `CURRENT_DATE - INTERVAL 7 DAYS`
4. Click on `{{end_date}}`
5. **Type**: Date
6. **Default**: `CURRENT_DATE`

**Visualization:** Counter/Scorecard showing total readings

### Filter 2: Multi-Select Model Categories

Allow filtering by multiple device models:


In [0]:
# Query: model_comparison
# Multi-select filter

query_with_multiselect = """
SELECT 
  model_category,
  COUNT(DISTINCT factory_name) as factories,
  AVG(average_temperature) as avg_temp,
  SUM(active_devices) as total_devices
FROM default.db_crash_course.iot_sensor_metrics
WHERE model_category IN ({{model_filter}})
  AND reading_date >= CURRENT_DATE - INTERVAL 7 DAYS
GROUP BY model_category
ORDER BY total_devices DESC
"""

print("Query with multi-select filter:")
print(query_with_multiselect)
print("\n📋 Copy this query into a new SQL query")


**Setup Multi-Select Parameter:**

First, create parameter query:


In [0]:
# Query: get_model_categories

model_param_query = """
SELECT DISTINCT 
  model_category as value,
  model_category as label
FROM default.db_crash_course.iot_sensor_metrics
ORDER BY model_category
"""

print("Parameter query for model categories:")
print(model_param_query)


Then in your main query:

1. Click on `{{model_filter}}`
2. **Type**: Query Based Multi-Select Dropdown
3. **Query**: Select `get_model_categories`
4. **Allow multiple selections**: Yes

**Visualization:** Table showing all selected models

---

## 5. Putting It All Together

Your dashboard now has:

1. **Bar Chart** - Factory temperatures by region (static parameter)
2. **Line Chart** - Temperature trend over 30 days (query-based parameter)
3. **Counter** - Total readings in date range (date filters)
4. **Table** - Model comparison (multi-select filter)

### Dashboard Layout Tips

**Organize your dashboard:**
- Put filters at the top
- Key metrics (counters) in first row
- Trend charts in middle
- Detailed tables at bottom

**Add Text Boxes:**
- Click **Add** → **Text Box**
- Add headers like "Factory Performance Overview" or "Last Updated: {{NOW}}"

---

## Summary

In this notebook, you learned:

✅ **Static parameters** - Quick dropdown filters with fixed values  
✅ **Query-based parameters** - Dynamic lists from live data  
✅ **Date range filters** - Time-based data selection  
✅ **Multi-select filters** - Compare multiple categories  
✅ **Using semantic models** - Consistent metrics in dashboards  

### Key Takeaways:

1. **Static parameters are fastest** - Use for fixed lists (regions, statuses)
2. **Query-based parameters are dynamic** - Auto-update as data changes
3. **Semantic models ensure consistency** - Same metrics everywhere
4. **Start simple** - Add complexity only when needed

### What You Built:

An interactive dashboard with 4 visualizations and multiple filters - exactly what leadership needs to monitor factory performance!

---

## Try This Out:

Want to go further? Try these additions:

1. **Add a "critical readings" counter** - Count readings where temperature > 85°F
2. **Create an anomaly trend chart** - Show anomaly rate over time by region
3. **Schedule email updates** - Dashboard → Schedule → Daily at 8 AM
4. **Add conditional formatting** - Highlight factories with high temperatures in red
5. **Create drill-down links** - Click factory to see device-level details

---

**Additional Resources:**
- [Dashboard Parameters Tutorial](https://docs.databricks.com/aws/en/dashboards/tutorials/query-based-params)
- [Dashboard Filters](https://docs.databricks.com/aws/en/dashboards/filters)
- [Dashboard Best Practices](https://docs.databricks.com/aws/en/dashboards/)
- [Scheduling Dashboards](https://docs.databricks.com/aws/en/dashboards/schedule-subscribe)
